In [49]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from scipy import io
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import cv2
import os

In [50]:
train_images = list()
train_labels = None

test_images = list()
test_labels = None

image_files_path = "datasets/font_images/"
image_files = os.listdir(image_files_path)

image_count = len(image_files)
train_count = image_count*95//100

for i in range(train_count):
    file = image_files[i]
    train_images.append(cv2.imread(image_files_path + file, cv2.IMREAD_GRAYSCALE))

for i in range(train_count, image_count):
    file = image_files[i]
    test_images.append(cv2.imread(image_files_path + file, cv2.IMREAD_GRAYSCALE))

with open("datasets/font_labels.txt", "r") as f:
    lines = f.read().splitlines()

lines_int = list()

for i in lines:
    lines_int.append(ord(i) - 65)


train_labels = np.array(lines_int[:train_count])
test_labels  = np.array(lines_int[train_count:])

train_images = np.array(train_images)
test_images = np.array(test_images)

In [51]:
print(train_images.shape, test_images.shape, train_labels.shape, test_labels.shape)

(3408, 28, 28) (180, 28, 28) (3408,) (180,)


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding="same", use_bias="false", input_shape=(28,28,1)),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(32, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(64, (3,3), padding="same", use_bias="false"),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.35),
    layers.Dense(26, activation="softmax")
])

model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [56]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

checkpoint = ModelCheckpoint(filepath="checkpoints/epoch_{epoch:02d}.keras", save_weights_only=False, save_freq="epoch")

model.fit(train_images, train_labels, epochs=15,
          validation_data=(test_images, test_labels), callbacks=[checkpoint], batch_size=5)

# Best = 95.13% validation acc

Epoch 1/15
682/682 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.0346 - loss: 3.2613 - val_accuracy: 0.0333 - val_loss: 3.2581
Epoch 2/15
682/682 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0311 - loss: 3.2591 - val_accuracy: 0.0389 - val_loss: 3.2582
Epoch 3/15
682/682 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0340 - loss: 3.2591 - val_accuracy: 0.0333 - val_loss: 3.2581
Epoch 4/15
682/682 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0340 - loss: 3.2591 - val_accuracy: 0.0333 - val_loss: 3.2582
Epoch 5/15
682/682 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0267 - loss: 3.2591 - val_accuracy: 0.0333 - val_loss: 3.2582
Epoch 6/15
682/682 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0343 - loss: 3.2591 - val_accuracy: 0.0333 - val_loss: 3.2582
Epoch 7/15
682/682 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0302 - loss: 3.2591 - val_accuracy: 0.0333 - val_loss: 3.2582
Epoch 8/15
682/682 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.0282 - loss: 3.2592 - val_accuracy: 0.

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels - 1
test_labels = test_labels - 1

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

model = tf.keras.models.load_model("alpha_identifier.keras")
loss, acc = model.evaluate(test_images, test_labels, batch_size=64)
print("Accuracy: ", acc*100, " Loss: ", loss)

325/325 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.9513 - loss: 0.1702
Accuracy:  95.12500166893005  Loss:  0.17023393511772156


In [ ]:
img = test_images[143]
cv2.imshow("hdb", img)
cv2.waitKey(1000)
img = img.reshape((-1,28,28,1))
prediction = model.predict(img)
print(np.argmax(prediction), test_labels[143])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
0 [0]
